In [1]:
import os

import sqlalchemy as sa
from sqlalchemy import orm

In [2]:
pg_user = os.environ["POSTGRES_USER"]
pg_password = os.environ["POSTGRES_PASSWORD"]
pg_host = "localhost"
pg_db = os.environ["POSTGRES_DB"]
engine = sa.create_engine(
    f"postgresql://{pg_user}:{pg_password}@{pg_host}:5432/{pg_db}",
    pool_pre_ping=True,
)
db = orm.sessionmaker(autocommit=False, autoflush=False, bind=engine)()

In [3]:
# with engine.connect() as connection:
res = db.execute(sa.select(sa.text("* from application")))
res.mappings().all()[0]["Корневой ИД заявки"]

'5309140'

In [5]:
defect_category_name = "Уборка"
type_of_work_performed = "Засор устранен"
district_code = "508.0"


def apply_where_filters(
    statement: sa.sql.selectable.Select, filters: dict[str, str]
) -> sa.sql.selectable.Select:
    parameter_count = 0
    for column, parameter_value in filters.items():
        parameter_count += 1
        parameter_name = f"user_param_{str(parameter_count)}"
        if parameter_value:
            statement = statement.where(
                sa.text(f'"{column}" = :{parameter_name}').bindparams(
                    sa.bindparam(parameter_name, value=parameter_value, type_=sa.String)
                )
            )
    return statement


filters = {
    "Наименование категории дефекта": defect_category_name,
    "Вид выполненных работ": type_of_work_performed,
    "Код района": district_code,
}
statement = sa.select(sa.text("* from application"))
statement_count = sa.select(sa.text("count(*) from application"))
statement = apply_where_filters(statement, filters)
statement_count = apply_where_filters(statement_count, filters)
statement = statement.limit(10).offset(0)

res_raw = db.execute(statement)
res = res_raw.mappings().all()
res_count_raw = db.execute(statement_count)
res_count = res_count_raw.scalar_one()
res_count

9